In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn import cross_validation
from sklearn.externals import joblib
from sklearn import svm
from sklearn.metrics import accuracy_score
import pandas as pd
import os

In [11]:
def preprocess(X,y):
    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=42)


   ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                     stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)
    joblib.dump(vectorizer, 'vectorizer.pkl')

   ### feature selection, because text is super high dimensional and
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    joblib.dump(selector, 'selector.pkl')
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()
    return features_train_transformed, features_test_transformed, labels_train, labels_test

In [16]:
clas=[]
sent=[]
for dataflie in os.listdir("data"):
    print(dataflie)
    df=pd.read_csv("data/"+dataflie)
    clas=clas+df['Id'].tolist()[1:]
    sent=sent+df['Pregunta'].tolist()[1:]

alarma.csv
partido.csv
clima.csv
prendeapaga.csv


In [21]:
def cleanPhrase(phrase):
    return phrase.replace("!","").replace(".","").replace(",","").replace(":","").replace("¿","").replace("?","").lower()
sent=[cleanPhrase(sen) for sen in sent]

In [59]:
features_train, features_test, labels_train, labels_test=preprocess(sent,clas)

In [60]:
lin_svc = svm.LinearSVC(C=1).fit(features_train, labels_train)

In [61]:
print(accuracy_score(labels_test, lin_svc.predict(features_test)))

0.904761904762


In [62]:
joblib.dump(lin_svc,"svm.pkl")

['svm.pkl']